### Intrinsic SSP Dynamics

This is a research-oriented demo that shows how SSPs can be updated by applying a recurrent transformation in the semantic pointer space such that the SSP evolves as a dynamical system. This uses Nengo, a population of IF spiking neurons, and a lowpass synapse on the recurrent connection.

In [ ]:
%matplotlib inline

In [ ]:
import base64

from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import nengo
import nengo_spa as spa

from nengo.utils.progress import Progress, ProgressTracker
from nengo.utils.matplotlib import rasterplot
from nengo_extras.plot_spikes import preprocess_spikes

from ssp.pointers import BaseVectors

In [ ]:
d = 64
rng = np.random.RandomState(seed=0)
gen = BaseVectors(d, rng=rng)

In [ ]:
from nengo_spa.algebras import HrrAlgebra
algebra = HrrAlgebra()
X = spa.SemanticPointer(next(gen), algebra=algebra, name="X")
Y = spa.SemanticPointer(next(gen), algebra=algebra, name="Y")

In [ ]:
# here we initially define a dX and dY translation in discrete time
# (i.e., with respect to the unit of discrete time-steps)
dx = 0.15
dy = 0.05
dt = 0.005

# we then determine the corresponding linear transformation
# (still in discrete time)
dX = X**dx
dY = Y**dy
T = algebra.get_binding_matrix((dX * dY).v)

In [ ]:
# optionally, we convert this into a continuous-time system whose
# time-constant will be with respect to the given time-step
# from nengolib.signal import discrete2cont, LinearSystem
# A, B, _, _ = discrete2cont(
#     LinearSystem((T, 1, 1, 0), analog=False), dt=dt, method='zoh').ss
# assert np.allclose(B, 1/dt)

In [ ]:
# now, we apply "discrete principle 3" (Voelker, 2019; equation 5.32)
# to map the discrete-time transformation onto a discretized lowpass
# with the given tau as its time-constant
tau = 0.1
a = np.exp(-dt / tau)
W = (T - a*np.eye(d)) / (1 - a)

In [ ]:
# define an initial starting point for the SSP
x0 = -8
y0 = -2
ssp0 = X**x0 * Y**y0

In [ ]:
# define the Nengo model with two spiking ReLU (IF) neurons per
# dimension (potentially spiking more than once per time-step)
# with W on a filtered recurrent connection

omega = 0.1*d  # scales the number of spikes (SNR)
with nengo.Network() as model:
    ssp = nengo.networks.EnsembleArray(
        n_neurons=2,
        n_ensembles=d,
        encoders=[[+1], [-1]],
        gain=[omega/dt, omega/dt],
        bias=[0, 0],
        neuron_type=nengo.SpikingRectifiedLinear(),
    )
    conn = nengo.Connection(
        ssp.add_output('output', function=lambda x:x, solver=nengo.solvers.Lstsq()),
        ssp.input,
        transform=W,
        synapse=tau,
    )
    p = nengo.Probe(ssp.input, synapse=None)  # the input is already filtered
    p_neurons = nengo.Probe(ssp.add_neuron_output(), synapse=None)

with nengo.Simulator(model, dt=dt) as sim:
    # initialize the PSC to the initial location of the SSP
    # (analogous to starting of an oscillator at some particular point
    # in space). this is far more accurate than giving it a "kick"
    signal = sim.model.sig[conn.synapse]
    sim.signals[signal['_state_X']] = [ssp0.v]

    # initialize the voltages to be uniformly distributed so there's
    # no initial transient (i.e., the neurons are already "mixed")
    for ens in ssp.ensembles:
        assert isinstance(ens.neuron_type, nengo.SpikingRectifiedLinear)
        signal = sim.model.sig[ens.neurons]
        sim.signals[signal['voltage']] = rng.rand(ens.n_neurons)

    sim.run(0.5)

In [ ]:
# compute the region map
scale = 10
xs = np.linspace(-scale, scale, 150)
ys = np.linspace(-scale, scale, 150)

# cmap = sns.diverging_palette(150, 275, s=80, l=55, as_cmap=True)
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)

m = np.empty((len(ys), len(xs), d))
for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        m[j, i] = (X**x * Y**y).v

In [ ]:
# compute the similarity map for every time-step in parallel
sims = m.dot(sim.data[p].T)

In [ ]:
# render an animation for the simulation
fig = plt.figure(figsize=(6, 6))

ims = []
with ProgressTracker(
    True, Progress("Animating", "Animation", len(sim.trange()))
) as progress_bar:
    for i in range(len(sim.trange())):
        im = plt.imshow(
            sims[:, :, i],
            interpolation='none',
            extent=(xs[0], xs[-1], ys[0], ys[-1]),
            vmin=-1,
            vmax=1,
            cmap=cmap,
            animated=True,
        )

        ims.append([im])
        progress_bar.total_progress.step()

plt.xticks([])
plt.yticks([])
plt.gca().invert_yaxis()

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)

# plt.show()
plt.close()

In [ ]:
# HTML(ani.to_html5_video())

# save it as a a gif and also embed it in the notebook as HTML
fname = ".temp.gif"
ani.save(fname, writer='imagemagick')
gif = open(fname, "rb").read()
gif_base64 = base64.b64encode(gif).decode()
HTML('<img src="data:image/gif;base64,{0}" />'.format(gif_base64))

In [ ]:
# for reference, compute the expected final location
# based on the ideal dynamical system and then plot it

xt = x0 + dx * len(sim.trange())
yt = y0 + dy * len(sim.trange())
sspt = X**xt * Y**yt

for name, v in (("Ideal Start", ssp0.v),
                ("Ideal End", sspt.v)):
    ref = m.dot(v)

    plt.figure(figsize=(6, 6))
    plt.title(name)
    plt.imshow(
        ref,
        interpolation='none',
        extent=(xs[0], xs[-1], ys[0], ys[-1]),
        vmin=-1,
        vmax=1,
        cmap=cmap,
    )
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
# show rasterplot of spikes, avg. spikes per neuron
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
rasterplot(*preprocess_spikes(sim.trange(), sim.data[p_neurons], num=128))
plt.xlabel("Time")
plt.ylabel("Neuron Number")

plt.subplot(1, 2, 2)
plt.plot(
    np.sum(sim.data[p_neurons] * dt, axis=1) / d,
)
plt.xlabel("Time-step")
plt.ylabel("# Spikes per Dimension")
plt.show()